In [5]:
# train_runner_multi.py
import os, pickle, numpy as np, torch, matplotlib.pyplot as plt, pandas as pd
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from ID3QNE_deepQnet import Dist_DQN

with open("requiredFile.pkl", "rb") as f:
    D = pickle.load(f)

Xtr = D["X_train"].to_numpy()
Xte = D["X_test"].to_numpy()
Xnext_tr = D["Xnext_train"].to_numpy()
ytr = D["y_train"]
yte = D["y_test"]
Atr = D["Action_train"]
n_actions = D.get("nbins", 25)

exps = [
    {"name": "exp1_baseline",  "seed": 42, "gamma": 0.99, "lr": 1e-5, "epochs": 100},
    {"name": "exp2_gamma095",  "seed": 42, "gamma": 0.95, "lr": 1e-5, "epochs": 100},
    {"name": "exp3_higher_lr", "seed": 42, "gamma": 0.99, "lr": 3e-5, "epochs": 100},
    {"name": "exp4_seed7",     "seed": 7,  "gamma": 0.99, "lr": 1e-5, "epochs": 100},
]

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def plot_training_loss(log_path, out_png, title):
    losses = []
    with open(log_path, "r") as f:
        for line in f:
            if "Loss:" in line:
                try: losses.append(float(line.strip().split("Loss:")[-1]))
                except: pass
    plt.close("all")
    plt.figure(figsize=(6,4), constrained_layout=True)
    if losses: plt.plot(losses)
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(title); plt.grid(True, alpha=0.3)
    plt.tight_layout(); plt.savefig(out_png, dpi=200); plt.close()

def plot_train_action_and_survival(A_train, y_train, out_dir, exp_name):
    plt.close("all")
    plt.figure(figsize=(6,4), constrained_layout=True)
    counts = np.bincount(A_train, minlength=n_actions)
    plt.bar(range(len(counts)), counts)
    plt.xlabel("Historical Action (0..24)"); plt.ylabel("Count")
    plt.title(f"{exp_name} – Training Action Distribution")
    plt.grid(True, axis="y", alpha=0.3)
    out1 = os.path.join(out_dir, f"{exp_name}_train_action_distribution.png")
    plt.savefig(out1, dpi=200); plt.close()

    df = pd.DataFrame({"action": A_train, "surv": (y_train == 0).astype(int)})
    by = df.groupby("action")["surv"].mean()
    plt.figure(figsize=(6,4), constrained_layout=True)
    by.plot(kind="bar"); plt.ylim(0,1)
    plt.xlabel("Historical Action (0..24)"); plt.ylabel("Survival Rate")
    plt.title(f"{exp_name} – Training Survival by Action")
    plt.grid(True, axis="y", alpha=0.3)
    out2 = os.path.join(out_dir, f"{exp_name}_train_survival_by_action.png")
    plt.tight_layout(); plt.savefig(out2, dpi=200); plt.close()

def policy_figure(q_net, Xtest_np, out_png, exp_name):
    with torch.no_grad():
        q_vals = q_net(torch.tensor(Xtest_np, dtype=torch.float32))
        actions_pred = q_vals.argmax(dim=1).cpu().numpy()

    fluid = actions_pred % 5; vaso = actions_pred // 5; N = len(actions_pred)
    fluid_prop = np.bincount(fluid, minlength=5).astype(float) / max(N,1)
    vaso_prop  = np.bincount(vaso,  minlength=5).astype(float) / max(N,1)
    heat = np.zeros((5,5), float)
    for f,v in zip(fluid, vaso): heat[v,f]+=1.0
    heat_prop = heat / max(N,1)

    plt.close("all")
    fig = plt.figure(figsize=(15, 4.8), constrained_layout=True)

    ax1 = fig.add_subplot(1,3,1)
    ax1.bar(range(5), fluid_prop)
    ax1.set_xticks(range(5)); ax1.set_xticklabels([f"F{i}" for i in range(5)])
    ax1.set_ylim(0,1); ax1.set_xlabel("Fluid Bin (0..4)", fontsize=9)
    ax1.set_ylabel("Proportion", fontsize=9); ax1.tick_params(labelsize=9)
    ax1.set_title(f"{exp_name}\nFluid Distribution (Test, Proportion)", pad=10, fontsize=11)
    ax1.grid(True, axis="y", alpha=0.3)

    ax2 = fig.add_subplot(1,3,2)
    ax2.bar(range(5), vaso_prop)
    ax2.set_xticks(range(5)); ax2.set_xticklabels([f"V{i}" for i in range(5)])
    ax2.set_ylim(0,1); ax2.set_xlabel("Vasopressor Bin (0..4)", fontsize=9)
    ax2.set_ylabel("Proportion", fontsize=9); ax2.tick_params(labelsize=9)
    ax2.set_title(f"{exp_name}\nVasopressor Distribution (Test, Proportion)", pad=10, fontsize=11)
    ax2.grid(True, axis="y", alpha=0.3)

    ax3 = fig.add_subplot(1,3,3, projection="3d")
    _x = np.arange(5); _y = np.arange(5)
    _xx,_yy = np.meshgrid(_x,_y)
    x = _xx.ravel(); y = _yy.ravel(); z = np.zeros_like(x, dtype=float)
    dx = np.full_like(x, 0.6, float); dy = np.full_like(y, 0.6, float)
    dz = heat_prop.ravel()
    cmap = cm.get_cmap("coolwarm")
    norm = colors.Normalize(vmin=dz.min(), vmax=dz.max() if dz.max()>0 else 1.0)
    colors_bar = cmap(norm(dz))
    ax3.bar3d(x-0.3, y-0.3, z, dx, dy, dz, color=colors_bar, shade=True)
    ax3.set_xticks(range(5)); ax3.set_xticklabels([f"F{i}" for i in range(5)], fontsize=8)
    ax3.set_yticks(range(5)); ax3.set_yticklabels([f"V{i}" for i in range(5)], fontsize=8)
    ax3.set_zlabel("Proportion", fontsize=9)
    ax3.set_xlabel("Fluid Bin", fontsize=9); ax3.set_ylabel("Vasopressor Bin", fontsize=9)
    ax3.set_title(f"{exp_name}\n5×5 Action Heatmap (Test, Proportion)", pad=10, fontsize=11)
    mappable = cm.ScalarMappable(norm=norm, cmap=cmap); mappable.set_array([])
    cb = fig.colorbar(mappable, ax=ax3, fraction=0.046, pad=0.08)
    cb.set_label("Proportion", fontsize=9)

    fig.savefig(out_png, dpi=220); plt.close(fig)

def summarize_metrics(out_csv, rows):
    df = pd.DataFrame(rows)
    if os.path.exists(out_csv):
        base = pd.read_csv(out_csv); df = pd.concat([base, df], ignore_index=True)
    df.to_csv(out_csv, index=False)

summary_rows = []
for cfg in exps:
    name = cfg["name"]; epochs = cfg["epochs"]
    outdir = os.path.join("runs", name); ensure_dir(outdir)
    print(f"\n=== Running {name} (epochs={epochs}, gamma={cfg['gamma']}, lr={cfg['lr']}, seed={cfg['seed']}) ===")

    model = Dist_DQN(state_dim=Xtr.shape[1], num_actions=n_actions,
                     gamma=cfg["gamma"], lr=cfg["lr"], seed=cfg["seed"])

    log_path = os.path.join(outdir, f"{name}_training_log.txt")
    open(log_path, "w").close()
    batchs = {"state": Xtr, "next_state": Xnext_tr, "action": Atr, "reward": np.where(ytr==0,24.0,-24.0)}

    for ep in range(epochs):
        loss = model.train_model(batchs, ep)
        print(f"{name} | Epoch {ep+1}/{epochs} | Loss: {loss:.4f}")
        with open(log_path, "a") as lf:
            lf.write(f"Epoch {ep+1} | Loss: {loss:.4f}\n")

    # save model
    model_path = os.path.join(outdir, f"{name}_model.pt")
    torch.save(model.q_net.state_dict(), model_path)

    # plots
    policy_png = os.path.join(outdir, f"{name}_policy_combined.png")
    policy_figure(model.q_net, Xte, policy_png, name)

    loss_png = os.path.join(outdir, f"{name}_training_loss.png")
    plot_training_loss(log_path, loss_png, f"{name} – Training Loss Over Time")

    plot_train_action_and_survival(Atr, ytr, outdir, name)

    # metrics
    sr_train = (ytr == 0).mean()
    sr_test  = (yte == 0).mean()
    er_train = np.where(ytr == 0, 24.0, -24.0).mean()
    er_test  = np.where(yte == 0, 24.0, -24.0).mean()

    summary_rows.append({
        "experiment": name, "epochs": epochs,
        "gamma": cfg["gamma"], "lr": cfg["lr"], "seed": cfg["seed"],
        "survival_train": sr_train, "survival_test": sr_test,
        "exp_return_train": er_train, "exp_return_test": er_test
    })

summarize_metrics("runs/summary_metrics.csv", summary_rows)
print("\nAll 4 experiments complete. Results in runs/exp*/ and runs/summary_metrics.csv")



=== Running exp1_baseline (epochs=100, gamma=0.99, lr=1e-05, seed=42) ===


TypeError: Dist_DQN.__init__() got an unexpected keyword argument 'lr'